In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
from plot_utils import *
from itertools import product, combinations, permutations

from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants
from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant


In [2]:
def generate_scenes(env, final_dir, data_dir, early_stop=None):
    outputs = []
    scene_index = 0
    with open(f"{data_dir}/index.jsonl", "w") as f:
        for background in trange(4):
            for quadrant in background_quadrants[background]:
                for objects in object_pairs:
                    for colors_pair in color_pairs:
                        for size in potential_sizes:
                            for directions in moving_directions:
                                for distance in direction_lengths:
                                    for steps in step_counts:
                                        scene_index += 1
                                        if early_stop is not None and scene_index > early_stop:
                                            return outputs
                                        
                                        configs = []
                                        for obj_index, obj_type in enumerate(objects):
                                            color = colors_pair[obj_index]
                                            random.seed(scene_index*1000+obj_index)
                                            position_range = get_position_range(background_quadrants, background, scale=0.6)
                                            if obj_type == 'custom':
                                                obj_config = create_config(obj_type, color_maps[color], size*1.1/2, position_range, name="002_master_chef_can")
                                            else:
                                                obj_config = create_config(obj_type, color_maps[color], size, position_range, name=obj_type)
                                            configs.append(obj_config)
                                        
                                        
                                        initilization_poses, mov_pos, ref_pos = initialize_obj_nooverlap_path(directions[0], configs, distance, env)
                                        initilization_poses[0] = mov_pos 
                                        
                                        env.set_poses(initilization_poses)

                                        # initialization_poses = env.get_important_obj_poses(mode="all")

                                        # env.set_poses(initialization_poses)
                                        
                                        direction_configs = [(0, 0, direction) for direction in directions]
                                        distances = [distance for _ in directions]
                                        frames = env.move_in_directions(direction_configs, distances, steps=steps, camera_view="all")
                                        
                                        # video_name = f"scene_{scene_index}"
                                        # video_path = os.path.join(final_dir, f"videos/{video_name}.mp4")
                                        # os.makedirs(os.path.dirname(video_path), exist_ok=True)
                                        # generate_video(video_path, frames['top0'], fps=20, color_change=True)

                                        image_dir = os.path.join(final_dir, f"images/scene_{scene_index}")
                                        save_images_pararell(frames['top0'], image_dir)
                                        image_dir_write = image_dir.replace("../", "")

                                        speed = distance / steps
                                        out_case = {
                                            "scene_index": scene_index,
                                            "source": image_dir_write,
                                            "moving_obj": colors_pair[0],
                                            "moving_obj_type": objects[0],
                                            "ref_obj": colors_pair[1],
                                            "ref_obj_type": objects[1],
                                            "direction": directions[0],
                                            "distance": distance,
                                            "steps": steps,
                                            "speed": speed,
                                            "size": size,
                                            "background": background,
                                            "quadrant": quadrant,
                                            #"initial_positions": [obj_config['ranges'] for obj_config in configs],
                                        }

                                        json.dump(out_case, f)
                                        f.write("\n")
                                        
                                        outputs.append(out_case)

    print(f"Generated {scene_index} scenes")
    return outputs

# Define directories
data_dir = "../benchmark/motion/speed"
final_dir = create_directories(data_dir, empty=True)

# Initialize parameters
random.seed(42)
objects = ["cube", "sphere"]
object_pairs = list(combinations(objects, 2))
moving_directions = [["left"], ["right"], ["front"], ["behind"]]
color_keys = list(color_maps.keys())
color_pairs = list(permutations(color_keys, 2))
random.shuffle(color_pairs)
color_pairs = color_pairs[:4]
direction_lengths = [0.2, 0.4]
potential_sizes = [0.03, 0.05]
step_counts = [10, 20]  # Three different step counts for three speeds

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    # Set early_stop to None if you want to generate all scenes
    early_stop = 10000
    outputs = generate_scenes(env, final_dir, data_dir, early_stop)

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-09-10 16:03:26.517] [svulkan2] [error] GLFW error: X11: The DISPLAY environment variable is missing
[2024-09-10 16:03:26.517] [svulkan2] [warning] Continue without GLFW.
  0%|          | 0/4 [00:00<?, ?it/s]